In [1]:
"""Checking a portfolio for transaction updates since an AsAt time


Attributes
----------
AsAt
Transaction Portfolios
GetPortfolioChanges API
"""

'Checking a portfolio for transaction updates since an AsAt time\n\n\nAttributes\n----------\nAsAt\nTransaction Portfolios\nGetPortfolioChanges API\n'

# 1. Introduction

In this notebook, we show how you can call LUSID to check if there have been any new, amended, or canceled transactions since a given `AsAt` datetime. From a technical point-of-view this is a two step process:

1. Call [GetPortfolioChanges](https://www.lusid.com/docs/api/#operation/GetPortfolioChanges) endpoint for a given `scope` and `AsAt` date. From that response, we get a list of all portfolios in that scope which have changes after the AsAt date.
2. Call [GetTransactions](https://www.lusid.com/docs/api/#operation/GetTransactions) on the list of portfolios from step #1, filtering for Transactions with a TransactionEntryDate after the asAt Date

> NOTE: We implement a two step check here with <b>GetPortfolioChanges</b> as we don't want to repeatedly call <b>GetTransactions</b> for every portfolio in a scope if there are no changes to be fetched.

![GetPortfolioChanges1](img/get-portfolio-changes/get-portfolio-changes.gif)

# 2. Setup LUSID

In [2]:
# Import general purpose packages
import os
import json
from datetime import datetime, timedelta, date
import pytz
from time import sleep

# Import lusid specific packages
import lusid
import lusid.models as models
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.seed_sample_data import seed_data
from lusidtools.cocoon.utilities import create_scope_id

# Import data wrangling packages
import pandas as pd

pd.set_option("display.max_columns", None)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

In [3]:
# Define the transaction portfolio API

txn_port_api = api_factory.build(lusid.api.TransactionPortfoliosApi)
entities_api = api_factory.build(lusid.api.EntitiesApi)
portfolios_api = api_factory.build(lusid.api.PortfoliosApi)

In [4]:
# Load a mapping file for DataFrame headers for the get holdings response
with open(r"config/get_holdings_mapping.json") as mappings_file:
    get_holdings_json_mapping = json.load(mappings_file)

# Load a mapping file for DataFrame headers for the get transactions response
with open(r"config/get_transactions_mapping.json") as mappings_file:
    get_txn_json_mapping = json.load(mappings_file)

In [5]:
def datetime_to_filter_string(entry_datetime, seconds_offset=0):
    
    date_obj = entry_datetime - timedelta(seconds=seconds_offset)
    
    return date_obj.isoformat()[:-6] + "0Z"

# 3. Create two sample portfolio for demonstration

Here we create two portfolios for demonstration. The portfolios are both UK equity portfolios holding FTSE 100 stocks.

In [6]:
# Create a new scope

scope = create_scope_id()
portfolio_code1 = "EQUITY_UK1"
portfolio_code2 = "EQUITY_UK2"

In [7]:
# Load a file of equity transactions

transactions_file = r"data/get-portfolio-changes/equity_transactions.csv"
transactions_df = pd.read_csv(transactions_file)

In [8]:
transactions_df.head(2)

,portfolio_code,portfolio_name,portfolio_base_currency,ticker,sedol,instrument_type,instrument_id,name,txn_id,txn_type,txn_trade_date,txn_settle_date,txn_units,txn_price,txn_consideration,currency,strategy,cash_transactions
0,EQUITY_UK1,LUSID's top 10 FTSE stock portfolio,GBP,GB0002162385,SEDOL1,equity,EQ_1234,Aviva,trd_0001,Buy,02/01/2020,04/01/2020,120000,5,600000,GBP,ftse_tracker,NaN
1,EQUITY_UK1,LUSID's top 10 FTSE stock portfolio,GBP,GB0002162385,SEDOL1,equity,EQ_1234,Aviva,trd_0002,Buy,02/01/2020,04/01/2020,12000,5,60000,GBP,ftse_tracker,NaN


In [9]:
# Load portfolios, instruments, and transactions

seed_data_response = seed_data(
    api_factory,
    ["portfolios", "instruments", "transactions"],
    scope,
    transactions_file,
    "csv",
)

In [10]:
response = txn_port_api.get_holdings(
    scope=scope, code=portfolio_code1, property_keys=["Instrument/default/Name"]
)

holdings_df = lusid_response_to_data_frame(
    response, rename_properties=True, column_name_mapping=get_holdings_json_mapping
)

holdings_df

,instrument_scope,LusidInstrumentId,SubHoldingKeys,InstrumentName,SourcePortfolioId,SourcePortfolioScope(default-Properties),HoldingType,Units,SettledUnits,Amount-Cost,Currency-Cost,Amount-CostPortfolioCcy,Currenct-CostPortfolioCcy,currency
0,default,LUID_BM2Y3771,{},Aviva,EQUITY_UK1,3ad9-1a5c-d3f8-24,P,132000.0,132000.0,660000.0,GBP,660000.0,GBP,GBP
1,default,LUID_NQW6C8T9,{},BHP,EQUITY_UK1,3ad9-1a5c-d3f8-24,P,120000.0,120000.0,2160000.0,GBP,2160000.0,GBP,GBP
2,default,LUID_PNYMFTKH,{},Barclays,EQUITY_UK1,3ad9-1a5c-d3f8-24,P,300000.0,300000.0,600000.0,GBP,600000.0,GBP,GBP
3,default,LUID_SICKC6KS,{},BP,EQUITY_UK1,3ad9-1a5c-d3f8-24,P,200000.0,200000.0,1000000.0,GBP,1000000.0,GBP,GBP
4,default,LUID_58VOI6IF,{},HSBC,EQUITY_UK1,3ad9-1a5c-d3f8-24,P,40000.0,40000.0,240000.0,GBP,240000.0,GBP,GBP
5,default,CCY_GBP,{},CASH_GBP,EQUITY_UK1,3ad9-1a5c-d3f8-24,B,3260000.0,3260000.0,3260000.0,GBP,3260000.0,GBP,GBP
6,default,LUID_BO7OTI5K,{},Morrisons,EQUITY_UK1,3ad9-1a5c-d3f8-24,P,360000.0,360000.0,720000.0,GBP,720000.0,GBP,GBP
7,default,LUID_447EXYXY,{},Tesco,EQUITY_UK1,3ad9-1a5c-d3f8-24,P,12000.0,12000.0,100000.0,GBP,100000.0,GBP,GBP
8,default,LUID_CNYYI8CU,{},Rightmove,EQUITY_UK1,3ad9-1a5c-d3f8-24,P,160000.0,160000.0,960000.0,GBP,960000.0,GBP,GBP
9,default,LUID_O53ONFI9,{},vodafone,EQUITY_UK1,3ad9-1a5c-d3f8-24,P,900000.0,900000.0,900000.0,GBP,900000.0,GBP,GBP


We sleep for 1 second to create a short break in the asat timeline, so we can compare results below.

In [11]:
sleep(1)

# 4. Book a batch of fund flows for both portfolios

Next, we book a batch of fund flows into both funds. The first set of transactions are all backdated. We book some current and future dated transactions later.

In [12]:
# Define now in UTC
now = datetime.now(tz=pytz.UTC)

# Get SOD (midnight) today
today = date.today()
today_sod = pytz.utc.localize(datetime.combine(today, datetime.min.time()))

# Create upsert datetimes for yesterday
eight_pm_yesterday = today_sod - timedelta(hours=4)
ten_pm_yesterday = today_sod - timedelta(hours=2)

# Create upsert datetimes for today
nine_am_today = today_sod + timedelta(hours=9)
eleven_am_today = today_sod + timedelta(hours=11)

# Create upsert datetimes for tomorrow
nine_am_tomorrow = today_sod + timedelta(hours=33)

Define function to manage fund flow upsert:

In [13]:
def upsert_fund_flow(fund_flow_txns):
    
    """
    This function upserts a set of fund flows into LUSID. 
    We purposefully leave 2 seconds between each upsert. 
    The function returns a tuple containing:
    1. A list of upsert AsAt times
    2. A list of UpsertTransaction responses from LUSID
    
    """
    
    upsert_asat_times = []
    upsert_responses = []
    
    # Leave 2 seconds between each upsert so we can view changes
    sleep(2)
    
    for portfolio_code, txn_id, txn_type, currency, date, amount in fund_flow_txns:
        
        upsert_transactions = txn_port_api.upsert_transactions(
            scope=scope,
            code=portfolio_code,
            transaction_request=[
                models.TransactionRequest(
                    transaction_id=txn_id,
                    type=txn_type,
                    instrument_identifiers={"Instrument/default/LusidInstrumentId": f"CCY_{currency}"},
                    transaction_date=date,
                    settlement_date=date,
                    units=amount,
                    transaction_price=models.TransactionPrice(price=1, type="Price"),
                    total_consideration=models.CurrencyAndAmount(amount=amount, currency=currency),
                    properties={},
                )
            ],
        )
        
        upsert_as_at = upsert_transactions.version.as_at_date
        
        print(f"> Transaction {txn_id} in portfolio {portfolio_code} upserted AsAt: {upsert_as_at}")
        
        upsert_asat_times.append(upsert_as_at)
                
    return (upsert_asat_times, upsert_responses)

In [14]:
fund_flow_txns_batch_1 = [
    
    # Post transactions for portfolio 1
    (portfolio_code1, "funds_001", "FundsIn", "GBP", eight_pm_yesterday, 100000),
    (portfolio_code1, "funds_002", "FundsIn", "USD", ten_pm_yesterday, 300000),
    
    # Post transactions for portfolio 2
    (portfolio_code2, "funds_001", "FundsIn", "GBP", eight_pm_yesterday, 100000),
    (portfolio_code2, "funds_002", "FundsIn", "USD", ten_pm_yesterday, 300000)

]

Upsert the fund flow transactions:

In [15]:
upsert_funds_asat = upsert_fund_flow(fund_flow_txns_batch_1)[0]

> Transaction funds_001 in portfolio EQUITY_UK1 upserted AsAt: 2022-06-28 14:14:20.350064+00:00
> Transaction funds_002 in portfolio EQUITY_UK1 upserted AsAt: 2022-06-28 14:14:20.690228+00:00
> Transaction funds_001 in portfolio EQUITY_UK2 upserted AsAt: 2022-06-28 14:14:21.022818+00:00
> Transaction funds_002 in portfolio EQUITY_UK2 upserted AsAt: 2022-06-28 14:14:22.886621+00:00


There are two timestamps we need to capture:

1. First we need to capture the <b>time just before the upsert</b>. 
2. Second, we need to capture entry datetime from the last transaction in the upsert batch. We'll use this datetime below to filter for every transaction since the <b>last entry datetime from the previous batch</b>

In [16]:
# Datetime just before upsert
first_batch_upsert_time_minus_1_second = datetime_to_filter_string(upsert_funds_asat[0], seconds_offset=1)

# Datetime of last transaction
first_batch_last_upsert_time = datetime_to_filter_string(upsert_funds_asat[-1])


print(f"> Datetime of first transaction minus 1 second: {first_batch_upsert_time_minus_1_second}")
print(f"> Datetime of last transaction: {first_batch_last_upsert_time}")

> Datetime of first transaction minus 1 second: 2022-06-28T14:14:19.3500640Z
> Datetime of last transaction: 2022-06-28T14:14:22.8866210Z


# 5. Track the transaction updates since fund flow posting


In this section we retrieve all transactions which were posted <b>after</b> the first set of equity transactions were uploaded into LUSID. We use the EntryDateTime data point to query results.


![GetPortfolioChanges2](img/get-portfolio-changes/get-changes-1.gif)


In [17]:
def get_latest_changes_entered_since_asat_date(scope, last_checked_date):
    
    """
    This function returns all the transactions posted into all portfolios in a scope since 
    an AsAt time provided by the user.
    
    There is two step process:
    1. Call the GetPortfolioChanges API to get the list of portfolios with changes
    2. Call the GetTransactions API to get details of transactions for a given portfolio
    
    """
    
    # The max_date resolves to the year 9999
    # We use this as a proxy to check for any changes across all effective space
    
    max_date = datetime.max.isoformat() + "Z"
    
    
    def get_transactions_since_date(portfolio_code, last_checked_date):        
        
        api_filter = f"entryDateTime gt {last_checked_date}"
                                
        get_transactions = txn_port_api.get_transactions(
            scope=scope,
            code=portfolio_code,
            filter=api_filter)
        
        txn_df = lusid_response_to_data_frame(get_transactions, rename_properties=True, column_name_mapping=get_txn_json_mapping )
        
        txn_df=txn_df[get_txn_json_mapping.values()]
        
        return txn_df
    
    portfolios_with_changes =  [i.entity_id.code for i in entities_api.get_portfolio_changes(
        scope=scope, effective_at=max_date, as_at=last_checked_date).values]
    
    
    for portfolio in portfolios_with_changes:
        
        print(f"Fetching transactions for portfolio {portfolio} since AsAt time {last_checked_date}")
        
        txns_df = get_transactions_since_date(portfolio, last_checked_date)
        
        display(txns_df)
        
    

Track the changes in the portfolio since the first upsert:

In [18]:
get_latest_changes_entered_since_asat_date(scope, first_batch_upsert_time_minus_1_second)

Fetching transactions for portfolio EQUITY_UK2 since AsAt time 2022-06-28T14:14:19.3500640Z


,PortfolioCode,EntryDateTime,TransactionId,TransactionType,FundFlowCurrency,Units,TransactionDate
0,EQUITY_UK2,2022-06-28 14:14:21.022818+00:00,funds_001,FundsIn,CCY_GBP,100000.0,2022-06-27 20:00:00+00:00
1,EQUITY_UK2,2022-06-28 14:14:22.886621+00:00,funds_002,FundsIn,CCY_USD,300000.0,2022-06-27 22:00:00+00:00


Fetching transactions for portfolio EQUITY_UK1 since AsAt time 2022-06-28T14:14:19.3500640Z


,PortfolioCode,EntryDateTime,TransactionId,TransactionType,FundFlowCurrency,Units,TransactionDate
0,EQUITY_UK1,2022-06-28 14:14:20.350064+00:00,funds_001,FundsIn,CCY_GBP,100000.0,2022-06-27 20:00:00+00:00
1,EQUITY_UK1,2022-06-28 14:14:20.690228+00:00,funds_002,FundsIn,CCY_USD,300000.0,2022-06-27 22:00:00+00:00


# 6. Post new and corrections to fund flow transactions

In [19]:
fund_flow_txns_batch_2 = [
    
    # Correct a transaction in portfolio 1
    (portfolio_code1, "funds_001", "FundsIn", "GBP", eight_pm_yesterday, 150000),
    
    # Post transactions for portfolio 2
    (portfolio_code2, "funds_003", "FundsIn", "EUR", nine_am_today, 200000),
    (portfolio_code2, "funds_004", "FundsOut", "GBP", eleven_am_today, 200000),
    (portfolio_code2, "funds_005", "FundsIn", "GBP", nine_am_tomorrow, 200000)

]

In [20]:
fund_flow_upsert = upsert_fund_flow(fund_flow_txns_batch_2)

> Transaction funds_001 in portfolio EQUITY_UK1 upserted AsAt: 2022-06-28 14:14:25.994520+00:00
> Transaction funds_003 in portfolio EQUITY_UK2 upserted AsAt: 2022-06-28 14:14:26.281057+00:00
> Transaction funds_004 in portfolio EQUITY_UK2 upserted AsAt: 2022-06-28 14:14:26.549886+00:00
> Transaction funds_005 in portfolio EQUITY_UK2 upserted AsAt: 2022-06-28 14:14:26.896096+00:00


# 7. Get the latest transactions cut

We can see below that the latest cut only includes:

1. New transactions
2. Corrections to old transactions posted since previous call
    - For example, see the transaction below in EQUITY_UK1 which has an updated units of 150000

In [21]:
get_latest_changes_entered_since_asat_date(scope, first_batch_last_upsert_time)

Fetching transactions for portfolio EQUITY_UK2 since AsAt time 2022-06-28T14:14:22.8866210Z


,PortfolioCode,EntryDateTime,TransactionId,TransactionType,FundFlowCurrency,Units,TransactionDate
0,EQUITY_UK2,2022-06-28 14:14:26.281057+00:00,funds_003,FundsIn,CCY_EUR,200000.0,2022-06-28 09:00:00+00:00
1,EQUITY_UK2,2022-06-28 14:14:26.549886+00:00,funds_004,FundsOut,CCY_GBP,200000.0,2022-06-28 11:00:00+00:00
2,EQUITY_UK2,2022-06-28 14:14:26.896096+00:00,funds_005,FundsIn,CCY_GBP,200000.0,2022-06-29 09:00:00+00:00


Fetching transactions for portfolio EQUITY_UK1 since AsAt time 2022-06-28T14:14:22.8866210Z


,PortfolioCode,EntryDateTime,TransactionId,TransactionType,FundFlowCurrency,Units,TransactionDate
0,EQUITY_UK1,2022-06-28 14:14:25.994520+00:00,funds_001,FundsIn,CCY_GBP,150000.0,2022-06-27 20:00:00+00:00


# 8. Clean-up: Delete portfolios

In [22]:
for port in (portfolio_code1, portfolio_code2):

    portfolios_api.delete_portfolio(scope=scope, code=port)